In [5]:
import json
import cv2
import os

In [20]:
def coco_resize(src_img, src_json, dst_img, dst_json):
    # Resize parameters
    new_width = 800
    new_height = 800

    # Load the COCO annotations
    with open(src_json, "r") as f:
        coco_data = json.load(f)

    # Prepare output directory
    os.makedirs(dst_img, exist_ok=True)

    # Update image dimensions and annotations
    for image_info in coco_data["images"]:
        # Load image
        image_path = os.path.join(src_img, image_info["file_name"])
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading image: {image_path}")
            continue
        
        # Get original dimensions
        original_width = image_info["width"]
        original_height = image_info["height"]
        
        # Resize image
        resized_image = cv2.resize(image, (new_width, new_height))
        resized_image_path = os.path.join(dst_img, image_info["file_name"])
        cv2.imwrite(resized_image_path, resized_image)
        
        # Update image information
        image_info["width"] = new_width
        image_info["height"] = new_height

        # Update annotations for this image
        image_id = image_info["id"]
        for annotation in coco_data["annotations"]:
            if annotation["image_id"] == image_id:
                # Update bounding box
                x_min, y_min, width, height = annotation["bbox"]
                x_min = x_min * new_width / original_width
                y_min = y_min * new_height / original_height
                width = width * new_width / original_width
                height = height * new_height / original_height
                annotation["bbox"] = [x_min, y_min, width, height]


    # Save updated COCO annotations
    with open(dst_json, "w") as f:
        json.dump(coco_data, f, indent=4)


In [23]:
def transToYoloFormat(inDir, outDir):
    with open(inDir) as f:
        data = json.load(f)
    #categories = {cat['id']: cat['name'] for cat in data['categories']}
    image_info = {img['id']: img for img in data['images']}

    for ann in data['annotations']:
        # Extract info
        image_id = ann["image_id"]
        category_id = ann["category_id"]
        bbox = ann["bbox"]  # [x_min, y_min, width, height]
        
        image = image_info[image_id]
        image_width, image_height = image["width"], image["height"]
        
        # Normalize bounding box
        x_min, y_min, box_width, box_height = bbox
        center_x = (x_min + box_width / 2) / image_width
        center_y = (y_min + box_height / 2) / image_height
        norm_width = box_width / image_width
        norm_height = box_height / image_height
        
        # YOLO format line
        yolo_line = f"{category_id - 1} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}"
        
        # Save to .txt file
        txt_filename = os.path.join(outDir, f"{image['file_name'].split('.')[0]}.txt")
        with open(txt_filename, "a") as txt_file:
            txt_file.write(yolo_line + "\n")


In [22]:

coco_resize('RPC-Dataset/original/train2019', 'RPC-Dataset/original/instances_train2019.json', 'RPC-Dataset/resize_800/train', 'RPC-Dataset/resize_800/train.json')
coco_resize('RPC-Dataset/original/test2019', 'RPC-Dataset/original/instances_test2019.json', 'RPC-Dataset/resize_800/test', 'RPC-Dataset/resize_800/test.json')
coco_resize('RPC-Dataset/original/val2019', 'RPC-Dataset/original/instances_val2019.json', 'RPC-Dataset/resize_800/val', 'RPC-Dataset/resize_800/val.json')

In [24]:
#resize labels
transToYoloFormat('RPC-Dataset/resize_800/train.json', 'RPC-Dataset/yolo_resize_800/train/labels')
transToYoloFormat('RPC-Dataset/resize_800/test.json', 'RPC-Dataset/yolo_resize_800/test/labels')
transToYoloFormat('RPC-Dataset/resize_800/val.json', 'RPC-Dataset/yolo_resize_800/valid/labels')

In [25]:
# create yaml file
jsonFile = 'RPC-Dataset/resize_800/train.json'

outPath = os.path.abspath('RPC-Dataset/yolo_resize_800')

for phase in ['train', 'test', 'valid']:
    os.makedirs(os.path.join(outPath, phase, 'images'), exist_ok=True)
    os.makedirs(os.path.join(outPath, phase, 'labels'), exist_ok=True)


with open(jsonFile) as f:
    data = json.load(f)

categories = [cat['name'] for cat in data['categories']]
nc = len(categories)

with open(os.path.join(outPath, 'data.yaml'), 'w') as f:
    f.write('train: ' + os.path.join(outPath, 'train/images') + '\n')
    f.write('test: ' + os.path.join(outPath, 'test/images') + '\n')
    f.write('val: ' + os.path.join(outPath, 'valid/images') + '\n\n')
    f.write('nc: ' + str(nc) + '\n')
    f.write('names: ' + str(categories) + '\n')
